In [17]:
from llama_index.core.prompts import RichPromptTemplate
import os
from dotenv import load_dotenv 

from llama_index.llms.google_genai import GoogleGenAI
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.schema import TextNode

from llama_index.core import Settings, VectorStoreIndex

load_dotenv()

True

In [19]:
llm = GoogleGenAI(
    model="gemini-2.0-flash",
    # api_key="some key",  # uses GOOGLE_API_KEY env var by default
)
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding


embed_model = GoogleGenAIEmbedding(model_name="text-embedding-004")

In [20]:
text_to_sql_prompt_tmpl_str = """\
You are a SQL expert. You are given a natural language query, and your job is to convert it into a SQL query.

Here are some examples of how you should convert natural language to SQL:

{{ examples }}


Now it's your turn.

Query: {{ query_str }}
SQL: 
"""

In [22]:
example_nodes = [
    TextNode(
        text="Query: How many params does llama 2 have?\nSQL: SELECT COUNT(*) FROM llama_2_params;"
    ),
    TextNode(
        text="Query: How many layers does llama 2 have?\nSQL: SELECT COUNT(*) FROM llama_2_layers;"
    ),
]



index = VectorStoreIndex(nodes=example_nodes,embed_model=embed_model)

# Create retriever
retriever = index.as_retriever(similarity_top_k=1)

In [23]:

from llama_index.core.prompts import RichPromptTemplate


def get_examples_fn(**kwargs):
    query = kwargs["query_str"]
    examples = retriever.retrieve(query)
    return "\n\n".join(node.text for node in examples)


prompt_tmpl = RichPromptTemplate(
    text_to_sql_prompt_tmpl_str,
    function_mappings={"examples": get_examples_fn},
)

In [26]:
prompt = prompt_tmpl.format(
    query_str="i need to select the people with salary more than the 1000000"
)
print(prompt)

You are a SQL expert. You are given a natural language query, and your job is to convert it into a SQL query.

Here are some examples of how you should convert natural language to SQL:

Query: How many params does llama 2 have?
SQL: SELECT COUNT(*) FROM llama_2_params;


Now it's your turn.

Query: i need to select the people with salary more than the 1000000
SQL: 


In [27]:
response=llm.complete(prompt)
print(response.text)

```sql
SELECT * FROM people WHERE salary > 1000000;
```
